In [113]:
import pandas as pd 
import numpy as np
import random

random.seed(42)
np.random.seed(42)

In [150]:
# Function to generate subsets as tuples
def n_size_random_subset(s, length=1):
    return tuple(sorted(random.sample(s, random.randint(length, len(s)))))

def generate_unique_weights(df, subject_col, units_col, route_col, weight_col='WEIGHTS'):
    """
    Generate unique Dirichlet-distributed weights for each unique combination of 
    (SUBJECT_ID, UNITS, ROUTE) and assign them to the DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - subject_col (str): Column name for SUBJECT_ID
    - units_col (str): Column name for UNITS
    - route_col (str): Column name for ROUTE (tuple of ints)

    Returns:
    - pd.DataFrame: DataFrame with a new WEIGHTS column
    """
    # Ensure ROUTE column is hashable (tuple format)
    df[route_col] = df[route_col].apply(tuple)

    # Extract unique groups
    unique_groups = df.drop_duplicates(subset=[subject_col, units_col, route_col]).copy()

    # Generate Dirichlet weights for each unique group
    unique_groups['WEIGHTS'] = unique_groups[route_col].map(
        lambda route: tuple(np.round(np.random.dirichlet([1] * len(route), size=1),2)[0])
    )

    # Merge back to original DataFrame
    return df.merge(unique_groups, on=[subject_col, units_col, route_col], how='left')['WEIGHTS']
    
def generate_unique_sub_weights(df, subject_col, units_col, route_col):
    """
    Generate unique pairs of Dirichlet-distributed weights for each element in ROUTE, 
    ensuring that each pair sums to 1.

    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - subject_col (str): Column name for SUBJECT_ID
    - units_col (str): Column name for UNITS
    - route_col (str): Column name for ROUTE (tuple of ints)

    Returns:
    - pd.DataFrame: DataFrame with a new column containing pairs of Dirichlet weights summing to 1
    """
    # Ensure ROUTE column is hashable (tuple format)
    df[route_col] = df[route_col].apply(tuple)

    # Extract unique groups
    unique_groups = df.drop_duplicates(subset=[subject_col, units_col, route_col]).copy()

    # Generate Dirichlet weight pairs for each element in ROUTE
    def generate_pairs(route):
        n = len(route)  # Number of elements in ROUTE
        pair_weights = np.round(np.random.dirichlet([1, 1], size=n),2)  # Generate n pairs of (w1, w2)
        return tuple(map(tuple, pair_weights))  # Convert to tuple of tuples

    unique_groups['SUB_WEIGHTS'] = unique_groups[route_col].map(generate_pairs)

    # Merge back to original DataFrame
    return df.merge(unique_groups, on=[subject_col, units_col, route_col], how='left')['SUB_WEIGHTS']
    
def map_exam_weights(df, route_col, weights_col, sub_weights_col, exams_col, exam_weights_col='EXAMS_WEIGHTS', exam_sub_weights_col='EXAMS_SUB_WEIGHTS'):
    """
    Generate EXAM_WEIGHTS and EXAM_SUB_WEIGHTS by mapping EXAMS to their corresponding values in WEIGHTS and WEIGHTS_PAIRS.

    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - route_col (str): Column containing ROUTE (tuple of ints)
    - weights_col (str): Column containing WEIGHTS (tuple of weights)
    - weights_pairs_col (str): Column containing WEIGHTS_PAIRS (tuple of (w1, w2) pairs)
    - exams_col (str): Column containing EXAMS (subset of ROUTE)
    - exam_weights_col (str): Name of output column for EXAM_WEIGHTS
    - exam_sub_weights_col (str): Name of output column for EXAM_SUB_WEIGHTS

    Returns:
    - pd.DataFrame: Updated DataFrame with EXAM_WEIGHTS and EXAM_SUB_WEIGHTS.
    """
    def extract_exam_weights(row):
        """Extract weights for elements in EXAMS from WEIGHTS and WEIGHTS_PAIRS"""
        route = row[route_col]
        exams = row[exams_col]
        weights = row[weights_col]
        sub_weights = row[sub_weights_col]

        # Create a mapping from ROUTE to WEIGHTS and WEIGHTS_PAIRS
        route_to_weights = dict(zip(route, weights))
        route_to_sub_weights = dict(zip(route, sub_weights))

        # Extract corresponding weights for EXAMS
        exam_weights = tuple(route_to_weights[exam] for exam in exams if exam in route_to_weights)
        exam_sub_weights = tuple(route_to_sub_weights[exam] for exam in exams if exam in route_to_sub_weights)

        return exam_weights, exam_sub_weights

    # Apply function to extract exam weights and sub-weights
    df[[exam_weights_col, exam_sub_weights_col]] = df.apply(lambda row: pd.Series(extract_exam_weights(row)), axis=1)

    return df

In [157]:
# Adjusting the range to range(4)
random.seed(1)
np.random.seed(1)
data_small_range = []
for _ in range(500):
    route = n_size_random_subset(range(1,5),2)  # subsets from range(4)
    exams = n_size_random_subset(route, 1)  # subsets from ROUTE
    data_small_range.append((route, exams))

df = pd.DataFrame(data_small_range, columns=['ROUTE', 'EXAMS'])
df['ID'] = np.random.permutation(np.arange(100_000_000, 100000000 + len(df)))
df['SUBJECT_ID'] = np.random.choice([16, 35], size=len(df))
df['UNITS'] = np.random.choice([4, 5], size=len(df))

df['WEIGHTS'] = generate_unique_weights(df, subject_col='SUBJECT_ID', units_col='UNITS', route_col='ROUTE')
df['SUB_WEIGHTS'] = generate_unique_sub_weights(df, subject_col='SUBJECT_ID', units_col='UNITS', route_col='ROUTE')

df['GRADES'] = [
    tuple((np.random.randint(0, 101), np.random.randint(0, 101)) for _ in range(len(t))) 
    for t in df['EXAMS']
]
df['SUBJECT_ID'] = np.random.choice([16, 35], size=len(df))
df['UNITS'] = np.random.choice([4, 5], size=len(df))
df['MAPPING'] = df.groupby(['SUBJECT_ID', 'UNITS'])['ROUTE'].transform(lambda x: pd.factorize(x)[0])
# df['MAPPING'] = pd.factorize(pd.MultiIndex.from_frame(df[['SUBJECT_ID', 'UNITS', 'ROUTE']]))[0]
# df['MAPPING'] = pd.factorize(df[['SUBJECT_ID','UNITS','ROUTE']])[0]
df = df[['ID','SUBJECT_ID','UNITS','ROUTE','MAPPING','WEIGHTS','SUB_WEIGHTS','EXAMS','GRADES']]

df = map_exam_weights(df, route_col='ROUTE', weights_col='WEIGHTS', sub_weights_col='SUB_WEIGHTS', exams_col='EXAMS')
df

,ID,SUBJECT_ID,UNITS,ROUTE,MAPPING,WEIGHTS,SUB_WEIGHTS,EXAMS,GRADES
0,100000304,16,5,"(1, 2)",0,"(0.99, 0.01)","((0.73, 0.27), (0.59, 0.41))","(2,)","((5, 92),)"
1,100000340,16,5,"(2, 3, 4)",1,"(0.57, 0.33, 0.1)","((0.34, 0.66), (0.37, 0.63), (0.79, 0.21))","(2,)","((79, 82),)"
2,100000047,16,4,"(1, 2, 3)",0,"(0.4, 0.34, 0.26)","((0.82, 0.18), (0.71, 0.29), (0.75, 0.25))","(1, 2, 3)","((74, 40), (25, 68), (49, 82))"
3,100000067,16,5,"(1, 2, 3, 4)",2,"(0.01, 0.0, 0.8, 0.18)","((0.27, 0.73), (0.27, 0.73), (0.36, 0.64), (0....","(1,)","((86, 46),)"
4,100000479,16,5,"(1, 2)",0,"(0.0, 1.0)","((0.32, 0.68), (0.37, 0.63))","(2,)","((13, 21),)"
...,...,...,...,...,...,...,...,...,...
495,100000255,16,4,"(1, 2)",9,"(0.99, 0.01)","((0.73, 0.27), (0.59, 0.41))","(1, 2)","((75, 25), (35, 79))"
496,100000072,16,5,"(2, 3, 4)",1,"(0.76, 0.18, 0.06)","((0.15, 0.85), (0.49, 0.51), (0.64, 0.36))","(2, 4)","((62, 76), (76, 74))"
497,100000396,35,5,"(2, 3, 4)",8,"(0.68, 0.15, 0.17)","((0.96, 0.04), (0.96, 0.04), (0.22, 0.78))","(2, 3)","((70, 47), (78, 31))"
498,100000235,16,5,"(1, 4)",4,"(0.0, 1.0)","((0.45, 0.55), (0.16, 0.84))","(4,)","((33, 43),)"


In [46]:
import pandas as pd
import numpy as np

def pivot_data_routes(df: pd.DataFrame) -> pd.DataFrame:
    exploded = df.explode(['EXAMS', 'GRADES','SUB_WEIGHTS']).reset_index()
    grades = pd.DataFrame(exploded['GRADES'].tolist(), columns=['EXAM_GRADE', 'MAGEN_GRADE'])
    weights = pd.DataFrame(exploded['SUB_WEIGHTS'].tolist(), columns=['EXAM_WEIGHT', 'MAGEN_WEIGHT'])
    
    exploded = pd.concat([exploded, grades, weights], axis=1)

    pivot_grade1 = exploded.pivot(index='ID', columns='EXAMS', values='EXAM_GRADE')
    pivot_grade1.columns = [f"{col}_EXAM_GRADE" for col in pivot_grade1.columns]

    pivot_grade2 = exploded.pivot(index='ID', columns='EXAMS', values='MAGEN_GRADE')
    pivot_grade2.columns = [f"{col}_MAGEN_GRADE" for col in pivot_grade2.columns]

    pivot_grade3 = exploded.pivot(index='ID', columns='EXAMS', values='EXAM_WEIGHT')
    pivot_grade3.columns = [f"{col}_EXAM_WEIGHT" for col in pivot_grade3.columns]

    pivot_grade4 = exploded.pivot(index='ID', columns='EXAMS', values='MAGEN_WEIGHT')
    pivot_grade4.columns = [f"{col}_MAGEN_WEIGHT" for col in pivot_grade4.columns]

    return pd.concat([pivot_grade1, pivot_grade2, pivot_grade3, pivot_grade4], axis=1).sort_index(axis=1)

# Example usage:
np.random.seed(42)


pivot_df = pivot_data_routes(df)
display(pivot_df)


,1_EXAM_GRADE,1_EXAM_WEIGHT,1_MAGEN_GRADE,1_MAGEN_WEIGHT,2_EXAM_GRADE,2_EXAM_WEIGHT,2_MAGEN_GRADE,2_MAGEN_WEIGHT,3_EXAM_GRADE,3_EXAM_WEIGHT,3_MAGEN_GRADE,3_MAGEN_WEIGHT
ID,,,,,,,,,,,,
100000000,41.0,0.58,57.0,0.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,0.15,52.0,0.85
100000002,NaN,NaN,NaN,NaN,46.0,0.71,81.0,0.29,NaN,NaN,NaN,NaN
100000003,NaN,NaN,NaN,NaN,91.0,0.20,47.0,0.80,NaN,NaN,NaN,NaN
100000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.70,81.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...
100000495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,0.21,99.0,0.79
100000496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,0.09,73.0,0.91
100000497,42.0,0.78,10.0,0.22,65.0,0.10,85.0,0.90,NaN,NaN,NaN,NaN


In [65]:
# Function to compute FINAL_GRADE for each exam_id
def compute_final_grades(pivot_df: pd.DataFrame) -> pd.DataFrame:
    # Identify unique exam IDs in the columns
    exam_ids = set(col.split('_')[0] for col in pivot_df.columns if '_EXAM' in col)

    # Compute FINAL_GRADE for each exam_id separately
    for exam_id in exam_ids:
        exam_col = f"{exam_id}_EXAM_GRADE"
        magen_col = f"{exam_id}_MAGEN_GRADE"
        exam_weight_col = f"{exam_id}_EXAM_WEIGHT"
        magen_weight_col = f"{exam_id}_MAGEN_WEIGHT"

        # Ensure columns exist before calculation to avoid KeyErrors
        if all(col in pivot_df.columns for col in [exam_col, magen_col, exam_weight_col, magen_weight_col]):
            pivot_df[f"{exam_id}_FINAL_GRADE"] = (
                pivot_df[exam_col].fillna(0) * pivot_df[exam_weight_col].fillna(0) +
                pivot_df[magen_col].fillna(0) * pivot_df[magen_weight_col].fillna(0)
            )

    return pivot_df

# Compute FINAL_GRADE for each exam_id
pivot_df = compute_final_grades(pivot_df)

display(pivot_df)

,1_EXAM_GRADE,1_EXAM_WEIGHT,1_MAGEN_GRADE,1_MAGEN_WEIGHT,2_EXAM_GRADE,2_EXAM_WEIGHT,2_MAGEN_GRADE,2_MAGEN_WEIGHT,3_EXAM_GRADE,3_EXAM_WEIGHT,3_MAGEN_GRADE,3_MAGEN_WEIGHT,2_FINAL_GRADE,1_FINAL_GRADE,3_FINAL_GRADE
ID,,,,,,,,,,,,,,,
100000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.14,10.0,0.86,0.00,0.00,22.60
100000001,78.0,0.41,54.0,0.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,63.84,0.00
100000002,52.0,0.29,28.0,0.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,34.96,0.00
100000003,89.0,0.68,21.0,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,67.24,0.00
100000004,NaN,NaN,NaN,NaN,48.0,0.42,42.0,0.58,NaN,NaN,NaN,NaN,44.52,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000195,NaN,NaN,NaN,NaN,99.0,0.10,31.0,0.90,NaN,NaN,NaN,NaN,37.80,0.00,0.00
100000196,NaN,NaN,NaN,NaN,34.0,0.53,83.0,0.47,NaN,NaN,NaN,NaN,57.03,0.00,0.00
100000197,79.0,0.81,92.0,0.19,76.0,0.74,46.0,0.26,NaN,NaN,NaN,NaN,68.20,81.47,0.00


In [66]:
pivot_df.columns

Index(['1_EXAM_GRADE', '1_EXAM_WEIGHT', '1_MAGEN_GRADE', '1_MAGEN_WEIGHT',
       '2_EXAM_GRADE', '2_EXAM_WEIGHT', '2_MAGEN_GRADE', '2_MAGEN_WEIGHT',
       '3_EXAM_GRADE', '3_EXAM_WEIGHT', '3_MAGEN_GRADE', '3_MAGEN_WEIGHT',
       '2_FINAL_GRADE', '1_FINAL_GRADE', '3_FINAL_GRADE'],
      dtype='object')

In [43]:
main(df, 35, 5) 

creating model for (subject_id=35, units=5, mapping=0)
creating model for (subject_id=35, units=5, mapping=1)
creating model for (subject_id=35, units=5, mapping=2)
creating model for (subject_id=35, units=5, mapping=3)
creating model for (subject_id=35, units=5, mapping=4)
creating model for (subject_id=35, units=5, mapping=5)
creating model for (subject_id=35, units=5, mapping=6)


In [42]:
def get_filtered_data(df:pd.DataFrame, subject_id, units):
    return df[(df['SUBJECT_ID']==subject_id)&(df['UNITS']==units)]

def create_model(data: pd.DataFrame):
    data 

# for converting dicts
#  df['tuples_col'] = df['dict_col'].apply(lambda d: tuple(d.values()))

def main(complete_data_routes, subject_id, units):
    # filter people to SUBJECT_ID and UNITS
    pop = complete_data_routes[(complete_data_routes['SUBJECT_ID']==subject_id)&(complete_data_routes['UNITS']==units)]
    for mapping in pop.MAPPING.unique():
        mapping = int(mapping)
        print(f'creating model for ({subject_id=}, {units=}, {mapping=})')
        pop_filtered = pop[pop['MAPPING']==mapping]
        model = create_model(pop_filtered)